In [1]:
#import libraries
import pandas as pd
import numpy as np
import re

In [2]:
#open factories data
dff = pd.read_csv("data/factories-mexico-total.csv",skiprows=4)

In [3]:
#show data
dff

,Periodos,"Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Total nacional /a /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Aguascalientes /a /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Baja California > Total /a /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Baja California > Ensenada /a /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Baja California > Mexicali /a /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Baja California > Tecate /a /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Baja California > Tijuana /f1 /b","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Coahuila de Zaragoza > Total /a /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Coahuila de Zaragoza > Acu�a /a /f1",...,"Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Tamaulipas > Total /a /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Tamaulipas > Matamoros /a /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Tamaulipas > Nuevo Laredo /a /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Tamaulipas > Reynosa /a /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Tamaulipas > Otros municipios /a /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Veracruz de Ignacio de la Llave /a /f1","Manufacturas > Industria manufacturera, maquila

In [4]:
#drop final rows
dff = dff.drop([174,175,176])

In [5]:
#untidy data
dff_melted = pd.melt(dff, id_vars=["Periodos"], var_name ="State, City", value_name ="Factories").copy()
dff_melted

,Periodos,"State, City",Factories
0,2007/07,"Manufacturas > Industria manufacturera, maquil...",5062.0
1,2007/08,"Manufacturas > Industria manufacturera, maquil...",5082.0
2,2007/09,"Manufacturas > Industria manufacturera, maquil...",5108.0
3,2007/10,"Manufacturas > Industria manufacturera, maquil...",5128.0
4,2007/11,"Manufacturas > Industria manufacturera, maquil...",5143.0
...,...,...,...
10435,2021/08,"Manufacturas > Industria manufacturera, maquil...",265.0
10436,2021/09,"Manufacturas > Industria manufacturera, maquil...",262.0
10437,2021/10,"Manufacturas > Industria manufacturera, maquil...",262.0
10438,2021/11,"Manufacturas > Industria manufacturera, maquil...",263.0


In [6]:
rowsf = dff_melted.shape[0]
rowsf

10440

In [7]:
dff_melted["State, City"][220]

'Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Aguascalientes /a /f1'

In [8]:
#clean data (parentesis)
for i in range(rowsf):
    dff_melted["State, City"][i]= re.sub(r'\([^)]*\)', '', dff_melted["State, City"][i])

<ipython-input-8-6d6319264d3c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff_melted["State, City"][i]= re.sub(r'\([^)]*\)', '', dff_melted["State, City"][i])


In [9]:
dff_melted["State, City"][220]

'Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n  > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > Aguascalientes /a /f1'

In [10]:
#clean data (other text)
dff_clean= dff_melted.replace("Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n  > Por entidad federativa > Establecimientos manufactureros > N�mero de establecimientos activos seg�n entidades federativas y municipios > ","",regex=True)
dff_clean= dff_clean.replace(" /a /f1","",regex=True)
dff_clean= dff_clean.replace(">",",",regex=True)
dff_clean

,Periodos,"State, City",Factories
0,2007/07,Total nacional,5062.0
1,2007/08,Total nacional,5082.0
2,2007/09,Total nacional,5108.0
3,2007/10,Total nacional,5128.0
4,2007/11,Total nacional,5143.0
...,...,...,...
10435,2021/08,Otras entidades federativas,265.0
10436,2021/09,Otras entidades federativas,262.0
10437,2021/10,Otras entidades federativas,262.0
10438,2021/11,Otras entidades federativas,263.0


In [11]:
#split columns
dff_clean["Month"] = dff_clean["Periodos"].str.split('/', expand=True)[1]
dff_clean["Year"] = dff_clean["Periodos"].str.split('/', expand=True)[0]
dff_clean["State"] = dff_clean["State, City"].str.split(',', expand=True)[0]
dff_clean["City"] = dff_clean["State, City"].str.split(',', expand=True)[1]
dff_clean.drop(["Periodos","State, City"], axis=1)

,Factories,Month,Year,State,City
0,5062.0,07,2007,Total nacional,None
1,5082.0,08,2007,Total nacional,None
2,5108.0,09,2007,Total nacional,None
3,5128.0,10,2007,Total nacional,None
4,5143.0,11,2007,Total nacional,None
...,...,...,...,...,...
10435,265.0,08,2021,Otras entidades federativas,None
10436,262.0,09,2021,Otras entidades federativas,None
10437,262.0,10,2021,Otras entidades federativas,None
10438,263.0,11,2021,Otras entidades federativas,None


In [12]:
#arrange columns
dff_clean = dff_clean[["Year","Month", "State","City", "Factories"]]
dff_clean

,Year,Month,State,City,Factories
0,2007,07,Total nacional,None,5062.0
1,2007,08,Total nacional,None,5082.0
2,2007,09,Total nacional,None,5108.0
3,2007,10,Total nacional,None,5128.0
4,2007,11,Total nacional,None,5143.0
...,...,...,...,...,...
10435,2021,08,Otras entidades federativas,None,265.0
10436,2021,09,Otras entidades federativas,None,262.0
10437,2021,10,Otras entidades federativas,None,262.0
10438,2021,11,Otras entidades federativas,None,263.0


In [13]:
dff_clean["State"].unique()

array(['Total nacional', 'Aguascalientes', 'Baja California ',
       'Coahuila de Zaragoza ', 'Chihuahua ', 'Ciudad de M�xico',
       'Durango', 'Guanajuato ', 'Jalisco ', 'M�xico ', 'Nuevo Le�n ',
       'Puebla ', 'Quer�taro ', 'San Luis Potos� ', 'Sonora ',
       'Tamaulipas ', 'Veracruz de Ignacio de la Llave', 'Yucat�n ',
       'Otras entidades federativas'], dtype=object)

In [14]:
#clean other characters
dff_clean = dff_clean.replace("M�xico","Mexico",regex=True)
dff_clean = dff_clean.replace("Le�n","Leon",regex=True)
dff_clean = dff_clean.replace("Quer�taro","Queretaro",regex=True)
dff_clean = dff_clean.replace("Potos�","Potosi",regex=True)
dff_clean = dff_clean.replace("Yucat�n","Yucatan",regex=True)


In [15]:
dff_clean["State"].unique()

array(['Total nacional', 'Aguascalientes', 'Baja California ',
       'Coahuila de Zaragoza ', 'Chihuahua ', 'Ciudad de Mexico',
       'Durango', 'Guanajuato ', 'Jalisco ', 'Mexico ', 'Nuevo Leon ',
       'Puebla ', 'Queretaro ', 'San Luis Potosi ', 'Sonora ',
       'Tamaulipas ', 'Veracruz de Ignacio de la Llave', 'Yucatan ',
       'Otras entidades federativas'], dtype=object)

In [16]:
dff_clean.sample(20)

,Year,Month,State,City,Factories
9967,2011,08,Yucatan,M�rida,46.0
5235,2008,10,Nuevo Leon,Total,638.0
10129,2010,08,Yucatan,Otros municipios,34.0
2508,2013,07,Chihuahua,Chihuahua,96.0
9210,2021,01,Tamaulipas,Nuevo Laredo,36.0
5929,2008,08,Nuevo Leon,San Nicol�s de los Garza,58.0
1917,2007,10,Coahuila de Zaragoza,Torre�n,69.0
7464,2020,07,Queretaro,Otros municipios,138.0
4823,2017,12,Mexico,Tlalnepantla de Baz,36.0
5484,2015,01,Nuevo Leon,Apodaca,186.0


In [17]:
dff_clean["City"].unique()

array([None, ' Total', ' Ensenada', ' Mexicali', ' Tecate',
       ' Tijuana /f1 /b', ' Acu�a', ' Ramos Arizpe', ' Saltillo',
       ' Torre�n', ' Otros municipios', ' Chihuahua', ' Ju�rez', ' Leon',
       ' Guadalajara', ' Zapopan', ' Tlalnepantla de Baz', ' Toluca',
       ' Apodaca', ' Guadalupe', ' Monterrey',
       ' San Nicol�s de los Garza', ' Santa Catarina', ' Puebla',
       ' Queretaro', ' San Luis Potosi', ' Hermosillo', ' Nogales',
       ' Matamoros', ' Nuevo Laredo', ' Reynosa', ' M�rida'], dtype=object)

In [18]:
#clean other characters
dff_clean = dff_clean.replace("Acu�a","Acuña",regex=True)
dff_clean = dff_clean.replace("Torre�n","Torreon",regex=True)
dff_clean = dff_clean.replace("Ju�rez","Juarez",regex=True)
dff_clean = dff_clean.replace("Nicol�s","Nicolas",regex=True)
dff_clean = dff_clean.replace("M�rida","Merida",regex=True)
dff_clean = dff_clean.replace("/f1 /b","",regex=True)


In [19]:
dff_clean["City"].unique()

array([None, ' Total', ' Ensenada', ' Mexicali', ' Tecate', ' Tijuana ',
       ' Acuña', ' Ramos Arizpe', ' Saltillo', ' Torreon',
       ' Otros municipios', ' Chihuahua', ' Juarez', ' Leon',
       ' Guadalajara', ' Zapopan', ' Tlalnepantla de Baz', ' Toluca',
       ' Apodaca', ' Guadalupe', ' Monterrey',
       ' San Nicolas de los Garza', ' Santa Catarina', ' Puebla',
       ' Queretaro', ' San Luis Potosi', ' Hermosillo', ' Nogales',
       ' Matamoros', ' Nuevo Laredo', ' Reynosa', ' Merida'], dtype=object)

In [20]:
#remove none values
for z in range(rowsf):
    if dff_clean["City"][z] == None:
        dff_clean["City"][z] = "Total"

<ipython-input-20-c0f4d7c67df7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff_clean["City"][z] = "Total"


In [21]:
#remove blank spaces at the begining and end
for a in range(rowsf):
    dff_clean["State"][a] = dff_clean["State"][a].rstrip().lstrip()
    dff_clean["City"][a] = dff_clean["City"][a].rstrip().lstrip()

<ipython-input-21-9687192012f2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff_clean["State"][a] = dff_clean["State"][a].rstrip().lstrip()
<ipython-input-21-9687192012f2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff_clean["City"][a] = dff_clean["City"][a].rstrip().lstrip()


In [22]:
dff_clean.sample(20)

,Year,Month,State,City,Factories
5674,2016,05,Nuevo Leon,Guadalupe,90.0
5893,2020,02,Nuevo Leon,Monterrey,52.0
4855,2020,08,Mexico,Tlalnepantla de Baz,34.0
7981,2020,02,San Luis Potosi,Otros municipios,66.0
8029,2009,08,Sonora,Total,253.0
2918,2018,09,Chihuahua,Otros municipios,67.0
5590,2009,05,Nuevo Leon,Guadalupe,82.0
1770,2010,01,Coahuila de Zaragoza,Saltillo,64.0
1707,2019,04,Coahuila de Zaragoza,Ramos Arizpe,83.0
8618,2015,03,Sonora,Otros municipios,88.0


In [23]:
#flag border states and cities
border = ["Baja California", "Coahuila de Zaragoza", "Chihuahua", "Nuevo Leon", "Tamaulipas", "Sonora"]
f = len(border)
dff_clean["Border"] = "NA"
for k in range(rowsf):
    p = 0
    for l in range(f):
        if dff_clean["State"][k] == border[l]:
            p = p +1
    if p > 0:
        dff_clean["Border"][k] = "Yes"
    else:
        dff_clean["Border"][k] = "No"

<ipython-input-23-03ed1279ecd5>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff_clean["Border"][k] = "No"
<ipython-input-23-03ed1279ecd5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff_clean["Border"][k] = "Yes"


In [24]:
dff_clean.sample(20)

,Year,Month,State,City,Factories,Border
1632,2013,01,Coahuila de Zaragoza,Ramos Arizpe,93.0,Yes
1006,2018,11,Baja California,Tecate,98.0,Yes
2922,2019,01,Chihuahua,Otros municipios,67.0,Yes
4569,2011,04,Mexico,Total,298.0,No
7462,2020,05,Queretaro,Otros municipios,138.0,No
6962,2007,09,Queretaro,Total,170.0,No
4289,2016,12,Jalisco,Zapopan,68.0,No
8183,2007,12,Sonora,Hermosillo,57.0,Yes
492,2019,07,Baja California,Total,927.0,Yes
4406,2012,03,Jalisco,Otros municipios,147.0,No


In [25]:
dff_clean.groupby("Border")["Border"].agg("count")

Border
No     5046
Yes    5394
Name: Border, dtype: int64

In [26]:
dff_clean.query("State == 'Baja California'")

,Year,Month,State,City,Factories,Border
348,2007,07,Baja California,Total,1013.0,Yes
349,2007,08,Baja California,Total,1011.0,Yes
350,2007,09,Baja California,Total,1012.0,Yes
351,2007,10,Baja California,Total,1013.0,Yes
352,2007,11,Baja California,Total,1017.0,Yes
...,...,...,...,...,...,...
1213,2021,08,Baja California,Tijuana,609.0,Yes
1214,2021,09,Baja California,Tijuana,599.0,Yes
1215,2021,10,Baja California,Tijuana,599.0,Yes
1216,2021,11,Baja California,Tijuana,599.0,Yes


In [27]:
#Export to CSV
dff_clean.to_csv("data/factories-mexico-refined.csv", index = False, encoding = "UTF8")